## 1. Assign every record in mobike dataset to the Polygon it belongs to.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import dask.dataframe as dd
import dask_geopandas as dg
import dask

D:\Users\great\Anaconda3\envs\py364\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [2]:
mobike_ori = pd.read_csv('mobike_nanjing_20170918-24.txt', encoding='utf-8',sep='\s+', header=None)

D:\Users\great\Anaconda3\envs\py364\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mobike_ori.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,MBK86206238681505730314773,45e985b3dcb859d70a4631fc88d88bd2,8620623868,2017-09-18,18:25:35.0,118.808,32.0524,2017-09-18,19:18:28.0,118.815084,32.062967,20170918.0
1,MBK86203240971505730314611,c2d73ddc923e4c21f857e5841212e849,8620324097,2017-09-18,18:25:47.0,118.733,31.9878,2017-09-18,18:32:08.0,118.730417,31.992373,20170918.0
2,MBK02565733401505730314662,0e11477c7d559fe7b780d68875357b96,256573340,2017-09-18,18:25:46.0,118.743,32.0934,2017-09-18,18:30:34.0,118.742263,32.085911,20170918.0
3,MBK02560370341505730314931,d50596c0f8d0695fafe6e54d6ab71feb,256037034,2017-09-18,18:25:44.0,118.75,32.0645,2017-09-18,18:28:39.0,118.748185,32.067683,20170918.0
4,MBK02560286211505730314713,4581a390b8193bce51598bd25518e976,256028621,2017-09-18,18:25:22.0,118.742,32.0449,2017-09-18,18:34:38.0,NaN,NaN,20170918.0


In [4]:
mobike_ori.shape

(3636631, 12)

In [5]:
# drop rows where time is NaN
mobike_ori.dropna(how='any',subset=[3,4,7,8],inplace=True)
mobike_ori.shape

(3636629, 12)

In [6]:
mobike_ori['start_time'] = mobike_ori[3] +' '+ mobike_ori[4]
mobike_ori['end_time'] = mobike_ori[7] +' '+ mobike_ori[8]
mobike_ori['start_time'] = mobike_ori['start_time'].astype('datetime64[ns]')
mobike_ori['end_time'] = mobike_ori['end_time'].astype('datetime64[ns]')

In [7]:
mobike_ori.drop(columns=[3,4,7,8,11],inplace=True)
mobike_ori.head()

,0,1,2,5,6,9,10,start_time,end_time
0,MBK86206238681505730314773,45e985b3dcb859d70a4631fc88d88bd2,8620623868,118.808,32.0524,118.815084,32.062967,2017-09-18 18:25:35,2017-09-18 19:18:28
1,MBK86203240971505730314611,c2d73ddc923e4c21f857e5841212e849,8620324097,118.733,31.9878,118.730417,31.992373,2017-09-18 18:25:47,2017-09-18 18:32:08
2,MBK02565733401505730314662,0e11477c7d559fe7b780d68875357b96,256573340,118.743,32.0934,118.742263,32.085911,2017-09-18 18:25:46,2017-09-18 18:30:34
3,MBK02560370341505730314931,d50596c0f8d0695fafe6e54d6ab71feb,256037034,118.75,32.0645,118.748185,32.067683,2017-09-18 18:25:44,2017-09-18 18:28:39
4,MBK02560286211505730314713,4581a390b8193bce51598bd25518e976,256028621,118.742,32.0449,NaN,NaN,2017-09-18 18:25:22,2017-09-18 18:34:38


In [8]:
mobike_ori.rename(index=str, columns={0: "record_id", 1: "user_id",2: "bike_id", 5:"start_lon", 6:"start_lat",9:"end_lon",10:"end_lat"},inplace=True)
mobike_ori.head()

,record_id,user_id,bike_id,start_lon,start_lat,end_lon,end_lat,start_time,end_time
0,MBK86206238681505730314773,45e985b3dcb859d70a4631fc88d88bd2,8620623868,118.808,32.0524,118.815084,32.062967,2017-09-18 18:25:35,2017-09-18 19:18:28
1,MBK86203240971505730314611,c2d73ddc923e4c21f857e5841212e849,8620324097,118.733,31.9878,118.730417,31.992373,2017-09-18 18:25:47,2017-09-18 18:32:08
2,MBK02565733401505730314662,0e11477c7d559fe7b780d68875357b96,256573340,118.743,32.0934,118.742263,32.085911,2017-09-18 18:25:46,2017-09-18 18:30:34
3,MBK02560370341505730314931,d50596c0f8d0695fafe6e54d6ab71feb,256037034,118.75,32.0645,118.748185,32.067683,2017-09-18 18:25:44,2017-09-18 18:28:39
4,MBK02560286211505730314713,4581a390b8193bce51598bd25518e976,256028621,118.742,32.0449,NaN,NaN,2017-09-18 18:25:22,2017-09-18 18:34:38


In [9]:
mobike_ori.shape

(3636629, 9)

In [10]:
#drop rows where coordinates are Nan
mobike = mobike_ori.dropna(how='any',subset=['start_lon','start_lat','end_lon','end_lat'])
mobike.shape

(3619115, 9)

In [11]:
mobike.dtypes

record_id             object
user_id               object
bike_id               object
start_lon             object
start_lat             object
end_lon              float64
end_lat              float64
start_time    datetime64[ns]
end_time      datetime64[ns]
dtype: object

In [12]:
zone = gpd.read_file('shp_file/zone_125m_wgs84.shp')

In [13]:
zone.head()

I am densified (5 elements)
I am densified (5 elements)


,start_id,end_id,geometry
0,0,0,"POLYGON ((118.521625158895 31.8900790484462, 1..."
1,1,1,"POLYGON ((118.5242678257038 31.89004738004776,..."
2,2,2,"POLYGON ((118.5269104883526 31.8900156567382, ..."
3,3,3,"POLYGON ((118.5295531468342 31.88998387851783,..."
4,4,4,"POLYGON ((118.5321958011414 31.88995204538694,..."


In [14]:
zone.crs

{'init': 'epsg:4326'}

In [15]:
mobike['start_lon'] = mobike['start_lon'].astype(float)
mobike['start_lat'] = mobike['start_lat'].astype(float)
mobike['record_id'] = mobike['record_id'].astype(str)
mobike['user_id'] = mobike['user_id'].astype(str)
mobike['bike_id'] = mobike['bike_id'].astype(str)
mobike.dtypes

D:\Users\great\Anaconda3\envs\py364\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Users\great\Anaconda3\envs\py364\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Users\great\Anaconda3\envs\py364\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

record_id             object
user_id               object
bike_id               object
start_lon            float64
start_lat            float64
end_lon              float64
end_lat              float64
start_time    datetime64[ns]
end_time      datetime64[ns]
dtype: object

In [16]:
mobike_dask = dd.from_pandas(mobike, npartitions=10)

In [17]:
mobike_dask_start = mobike_dask.set_geometry(mobike_dask[['start_lon','start_lat']], crs={'init' :'epsg:4326'})

In [18]:
type(mobike_dask_start)

dask_geopandas.core.GeoDataFrame

In [19]:
mobike_dask_start_1 = dg.sjoin(mobike_dask_start,zone[['start_id','geometry']])
full = mobike_dask_start_1[['record_id','user_id','bike_id','start_lon','start_lat','end_lon','end_lat','start_id','start_time','end_time']]
#full.to_parquet('test.parquet')
#full = dd.read_parquet('test.parquet')
#full.head()

D:\Users\great\Anaconda3\envs\py364\lib\site-packages\geopandas\tools\sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [20]:
type(full)

dask.dataframe.core.DataFrame

In [21]:
full = full.set_geometry(full[['end_lon','end_lat']], crs={'init' :'epsg:4326'})

In [22]:
mobike_dask_start_1 = dg.sjoin(full,zone[['end_id','geometry']])
full = mobike_dask_start_1[['record_id','user_id','bike_id','start_lon','start_lat','end_lon','end_lat','start_id','end_id','start_time','end_time']]

In [23]:
full.to_parquet('test.parquet')

In [24]:
aa = dd.read_parquet('test.parquet')

In [27]:
aa.head()

,record_id,user_id,bike_id,start_lon,start_lat,end_lon,end_lat,start_id,end_id,start_time,end_time
0,MBK86206238681505730314773,45e985b3dcb859d70a4631fc88d88bd2,8620623868,118.808255,32.052393,118.815084,32.062967,17616,18649,2017-09-18 18:25:35,2017-09-18 19:18:28
1,MBK86203240971505730314611,c2d73ddc923e4c21f857e5841212e849,8620324097,118.733422,31.987824,118.730417,31.992373,10584,10995,2017-09-18 18:25:47,2017-09-18 18:32:08
2,MBK86208648501505730313755,75c79a973762cc855b0a83520455121a,8620864850,118.737325,32.136352,118.739070,32.133887,26241,26036,2017-09-18 18:25:52,2017-09-18 18:30:36
3,MBK86204835351505730309626,844d89f4d8dc7d1cbe6930c12daac9d6,8620483535,118.811291,32.102423,118.840387,32.105020,22973,23190,2017-09-18 18:25:22,2017-09-18 18:41:40
4,MBK02500853931505730256250,f1959964a0dbe802d2bbcffde841ee81,250085393,118.797169,32.048483,118.789400,32.048150,17200,16992,2017-09-18 18:24:36,2017-09-18 18:33:02


In [26]:
aa.dtypes

record_id             object
user_id               object
bike_id               object
start_lon            float64
start_lat            float64
end_lon              float64
end_lat              float64
start_id               int64
end_id                 int64
start_time    datetime64[ns]
end_time      datetime64[ns]
dtype: object

### Things below didnt work out due to the large size of the dataset. it's very slow to do point in polygon query just using geopandas and shapely.

In [ ]:
# create GeoDataFrame from DataFrame. store start_point and end_point coordinates in MultiPoint as the geometry of the GeoDataFrame
#mobike_geo = gpd.GeoDataFrame(mobike, geometry = [MultiPoint([Point(x_1,y_1),Point(x_2,y_2)]) for x_1, y_1, x_2, y_2 in zip(mobike['start_lon'], mobike['start_lat'],mobike['end_lon'],mobike['end_lat'])])

In [ ]:
#print(mobike_geo.crs)

In [ ]:
def assign_point_to_polygon(row, start_point_index = 'start_point', end_point_index = 'end_point', grid_data = hexgon_grid.to_crs(epsg=4326), src_col='geometry'):
    grid_data.apply()
    for index, row_1 in grid_data.itertuples():
        start_lon = row['start_lon']
        start_lat = row['start_lat']
        end_lon = row['end_lon']
        end_lat = row['end_lat']
        # if record is beyond all polygons, set point id to 99999
        start_id = 99999
        end_id = 99999
        if row_1.contains(Point(start_lon,start_lat)):
            start_id = index
        if row_1.contains(Point(end_lon,end_lat)):
            end_id = index
    row[start_point_index] = start_id
    row[end_point_index] = end_id
    print(start_id)
    return row

In [ ]:
mobike = mobike.apply(assign_point_to_polygon,axis=1)